# Imports

In [167]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import sys
from neo4j import GraphDatabase
from neo4j.exceptions import CypherSyntaxError
from SPARQLWrapper import SPARQLWrapper, JSON  # pip install sparqlwrapper

# Defined functions

In [2]:
def get_sparql_results(endpoint_url, query):
    # Helper function
    user_agent = f"WDQS-example Python/{sys.version_info[0]}.{sys.version_info[1]}"  # https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

In [196]:
def scrape_qcodes(list, humans_only=False, first_only=False):
    """
    Gets Q codes and urls for wikidata pages from the first page of search results.
    
    list: list of search names, eg. ['Fryderyk Chopin', 'Ferenc Liszt', 'Wojciech Żywny']
    first_only: if True, gets data from the first link (result) only
    returns: list of dicts (search_name, page_name, description, Qcode, wiki_url)
             if page with given name doesn't exist, its qcode is ommited
    """
    retv = []
    input_length = len(list)-1
    for i, search_name in enumerate(list):
        print(f"Scraping... {i+1}/{input_length} ... " + search_name)
        url = f"https://www.wikidata.org/w/index.php?go=Go&search={search_name}"
        r = requests.get(url)
        soup = BeautifulSoup(r.text, "html.parser")
        for item in soup.find_all('li', class_="mw-search-result mw-search-result-ns-0"):
            page_name = item.find('a', title=True)['title'].lstrip('\u200e').split('\u200e')[0]
            try:
                description = item.find('a', title=True)['title'].lstrip('\u200e').split('\u200e')[2]
            except IndexError:
                description = None
            href = item.find('a', href=True).get('href')
            Qcode = href.lstrip('/wiki/')
            wiki_url = f"https://www.wikidata.org{href}"
            if humans_only:
                human_test = get_all_human_objects([Qcode])
                if human_test:
                    retv.append({'search_name': search_name, 'page_name': page_name, 'description':description, 'Qcode': Qcode, 'wiki_url': wiki_url,})
                    if first_only:
                        break
            else:
                retv.append({'search_name': search_name, 'page_name': page_name, 'description':description, 'Qcode': Qcode, 'wiki_url': wiki_url,})
                if first_only:
                    break
    return retv

In [4]:
def get_all_triples_and_qcodes(Qcode, allow_literals=True):
    """
    Args:
        Qcode: wikidata subject Qcode, eg.: 'Q255'
        allow_literals: whether to allow returning literal objects (objects not represented as wikidata/wikimedia pages) associated with subject's Qcode

    Returns (First value returned): 
        list of tuples representig RDF triples (object-predicate-subject) and labels associated with each wikidata entity

        Example tuple representing RDF triple:
            ('http://www.wikidata.org/entity/Q255',
            'http://www.wikidata.org/entity/P19',
            'http://www.wikidata.org/entity/Q586',
            'Ludwig van Beethoven',
            'place of birth',
            'Bonn')
        Tuple contents:
            (object,
            predicate,
            subject,
            objectLabel,
            predicateLabel,
            subjectLabel)

    Returns (Second value returned): 
        list of all Qcodes of wikidata objects associated with provided subject (Qcode)

        Example:
            ['Q255', Q'1268']
    """
    #  UWAGA:  Z jakiegoś powodu query poniżej zwraca wielokrotnie zduplikowane wyniki, pewnie błąd leży po stronie twórców biblioteki SPARQLWrapper.
    #          To samo zapytanie wprwadzowne w https://query.wikidata.org/ zwraca normalne, nie powielane wyniki.
    #          Poprawiłem to dopiero przy dodawaniu krotek do listy trójek RDF.
    endpoint_url = "https://query.wikidata.org/sparql"
    query = """SELECT DISTINCT ?s ?sLabel ?wd ?wdLabel ?o ?oLabel   WHERE {
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en,pl". }
    VALUES ?s { wd:%s }
    ?s ?wdt ?o.
    ?wd wikibase:directClaim ?wdt.
    ?wd rdfs:label ?wdLabel.
    FILTER((LANG(?wdLabel)) = "en")
    }
    ORDER BY (xsd:integer(STRAFTER(STR(?wd), "http://www.wikidata.org/entity/P")))"""%(Qcode) 
    results = get_sparql_results(endpoint_url, query)

    list_of_triples = []
    for result in results["results"]["bindings"]:
        # print(result['wdLabel']['value'])
        try:
            if result['wdLabel']['value'].__contains__('date'):  # np. Personendatenbank Germania Sacra tu wchodzi, ale to nie data...
                t = (result['s']['value'], result['wd']['value'], result['o']['datatype'], result['sLabel']['value'], result['wdLabel']['value'], result['oLabel']['value'])
        except KeyError: # Personendatenbank Germania Sacra... 
            pass  # Personendatenbank Germania Sacra... 
        if not allow_literals:
            if result['o']['type'] != 'literal':
                t = (result['s']['value'], result['wd']['value'], result['o']['value'], result['sLabel']['value'], result['wdLabel']['value'], result['oLabel']['value'])
        else:
            t = (result['s']['value'], result['wd']['value'], result['o']['value'], result['sLabel']['value'], result['wdLabel']['value'], result['oLabel']['value'])
        if t not in list_of_triples:  # <-- o tutaj
            list_of_triples.append(t)

    object_qcodes = []
    for t in list_of_triples:
        object_url = t[2]
        if "http://www.wikidata.org/entity/Q" in object_url:
            Qcode = object_url[object_url.rfind('Q'):]
            object_qcodes.append(Qcode)

    return list_of_triples, object_qcodes

In [5]:
def get_all_human_objects(list_of_qcodes):
    """
    Filters wikidata objects based on provided Qcodes. Returns only objects which are instance_of human.

    Args:
        list_of_qcodes example: ['Q255', Q'1268']
    
    Returns:
        list of tuples representing humans (qcode, wikidata object url, name)
    """
    #  UWAGA:  Z jakiegoś powodu query poniżej zwraca wielokrotnie zduplikowane wyniki, pewnie błąd leży po stronie twórców biblioteki SPARQLWrapper.
    #          To samo zapytanie wprwadzowne w https://query.wikidata.org/ zwraca normalne, nie powielane wyniki.
    #          Poprawiłem to przy przygotowywaniu zwracanego wyniku
    list_of_wdqcodes = ["wd:" + qcode for qcode in list_of_qcodes]
    query_check_objects = """SELECT ?qcode ?qcodeLabel WHERE {{
    SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en,pl". }}
    VALUES ?qcode {{ {} }}
    ?qcode wdt:P31 wd:Q5.
    }}""".format(' '.join(list_of_wdqcodes))
    endpoint_url="https://query.wikidata.org/sparql"
    humans = get_sparql_results(endpoint_url, query_check_objects)
    urls_qcodes_names = []
    for r in humans["results"]["bindings"]:
        url_qcode_and_name = (r['qcode']['value'][r['qcode']['value'].rfind('Q'):], r['qcode']['value'], r['qcodeLabel']['value'])
        if url_qcode_and_name not in urls_qcodes_names:
            urls_qcodes_names.append(url_qcode_and_name)
    return urls_qcodes_names
    

In [56]:
def recursive_get_rdf_info(input_qcodes, level=3):
    """For each provided qcode gets all rdf triples from wikidata. Also gets all rdf triples for humans associated with provided qcodes.
    If level=2, then also gets all rdf triples for humans associated with humans associated provided qcodes, etc...

    Args:
        input_qcodes (List): list of qcodes
        level (int, optional): "Depth of the recursive tree". Defaults to 3.

    Returns:
        Dict: Dictionary with qcodsccanocfdsgjwkdbfj:)lk]dsgdsa
    """
    rdf_dict = dict()
    current_humans = []
    for Q in input_qcodes:
        triples, all_qcodes = get_all_triples_and_qcodes(Q, allow_literals=True)
        rdf_dict[Q] = triples
        tuples = get_all_human_objects(all_qcodes)
        humans = [t[0] for t in tuples]  # human qcodes associated with Q qcode
        for new_qcode in humans:
            if new_qcode not in current_humans:
                current_humans.append(new_qcode)
            triples, all_qcodes = get_all_triples_and_qcodes(new_qcode, allow_literals=True)
            rdf_dict[new_qcode] = triples
    if level > 1:
        print("Done: ", input_qcodes)
        return recursive_get_rdf_info(current_humans, level=level-1)
    else:  # level == 1
        print("Done: ", input_qcodes)
        return rdf_dict

In [57]:
def recursive_get_rdf_info_v2_TO_CHECK(input_qcodes, level=3):
    """For each provided qcode gets all rdf triples from wikidata. Also gets all rdf triples for humans associated with provided qcodes.
    If level=2, then also gets all rdf triples for humans associated with humans associated provided qcodes, etc...

    Args:
        input_qcodes (List): list of qcodes
        level (int, optional): "Depth of the recursive tree". Defaults to 3.

    Returns:
        Dict: Dictionary with qcodsccanocfdsgjwkdbfj:)lk]dsgdsa
    """
    rdf_dict = dict()
    current_humans = []
    for Q in input_qcodes:
        if Q not in rdf_dict.keys():
            triples, all_qcodes = get_all_triples_and_qcodes(Q, allow_literals=True)
            rdf_dict[Q] = triples
            tuples = get_all_human_objects(all_qcodes)
            humans = [t[0] for t in tuples]  # human qcodes associated with Q qcode
        else:
            humans = []
        for new_qcode in humans:
            if new_qcode not in current_humans:
                current_humans.append(new_qcode)
            if new_qcode not in rdf_dict.keys():
                triples, all_qcodes = get_all_triples_and_qcodes(new_qcode, allow_literals=True)
                rdf_dict[new_qcode] = triples
    if level > 1:
        print("Done: ", input_qcodes)
        return recursive_get_rdf_info_v2_TO_CHECK(current_humans, level=level-1)
    else:  # level == 1
        print("Done: ", input_qcodes)
        return rdf_dict

# Input: List of persons ```(names from .txt file)```

In [8]:
with open("unique_persons.txt", 'r') as f:
    input = [name for name in f.read().splitlines() if len(name.split())!=1]
# input = ['Beethoven', 'Rachmaninoff', 'Mahler', 'Delta rzeki Okawango', 'Susanne Gibson-Milne', 'asfasfafasfs', 'sekwoja', 'Pierre']
# input = ['Rachmaninov']

In [22]:
print(len(input))
input

399


['Antoni Kątski',
 'Emilia Chopin',
 'Louis Boulanger',
 'Godefroi de Bouillon',
 'Peter Paul Rubens',
 'Susanne Gibson-Milner',
 'Ernest Canut',
 'Augustine Brault',
 'Izabella Działyńska (hrabina)',
 'Marie d’Agoult',
 'Amélie Franchomme',
 'Kazimierz Wernik',
 'Josef Dessauer',
 'Vera Rubio',
 'Franciszek Skibicki',
 'François-Victor Hugo',
 'Sabine Heinefetter',
 'Thomas Bohier',
 'Aleksander Colonna-Walewski (książę)',
 'Francesco de Vico',
 'Urbain Jean-Joseph Le Verrier',
 'Augustine Marie Bertholdi',
 'Fontana Juanna Camila',
 'Václav Hanka',
 'Ludwika Magdalena Ciechomska',
 'Gustaw Planche',
 'Giulia Grisi',
 'François-Joseph Gossec',
 'Honorata Dziewanowska',
 'Antoine de Orléans',
 'Jan Křtitel Pišek',
 'Władysław Żeleński',
 'Adolphe d Ennery',
 'Władysław Czartoryski',
 'Henryk Jędrzejewicz',
 'Mikołaj Chopin',
 'Aleksandra Karasowska',
 'Jules Janin',
 'Sigismond von Neukomm',
 'Thérèse Albrecht',
 'Adam Towiański',
 'Jean-Baptiste-Antoine Lassus',
 'Adam Mickiewicz',
 '

# Q-codes for names from input_list ```(Q-codes from wikidata search page)```

In [23]:
original_persons = scrape_qcodes(list=input, humans_only=True, first_only=True)  # 8m 30s dla 399 osób z listów z pliu .txt

Scraping... 0/399 ... Antoni Kątski
Scraping... 1/399 ... Emilia Chopin
Scraping... 2/399 ... Louis Boulanger
Scraping... 3/399 ... Godefroi de Bouillon
Scraping... 4/399 ... Peter Paul Rubens
Scraping... 5/399 ... Susanne Gibson-Milner
Scraping... 6/399 ... Ernest Canut
Scraping... 7/399 ... Augustine Brault
Scraping... 8/399 ... Izabella Działyńska (hrabina)
Scraping... 9/399 ... Marie d’Agoult
Scraping... 10/399 ... Amélie Franchomme
Scraping... 11/399 ... Kazimierz Wernik
Scraping... 12/399 ... Josef Dessauer
Scraping... 13/399 ... Vera Rubio
Scraping... 14/399 ... Franciszek Skibicki
Scraping... 15/399 ... François-Victor Hugo
Scraping... 16/399 ... Sabine Heinefetter
Scraping... 17/399 ... Thomas Bohier
Scraping... 18/399 ... Aleksander Colonna-Walewski (książę)
Scraping... 19/399 ... Francesco de Vico
Scraping... 20/399 ... Urbain Jean-Joseph Le Verrier
Scraping... 21/399 ... Augustine Marie Bertholdi
Scraping... 22/399 ... Fontana Juanna Camila
Scraping... 23/399 ... Václav Han

In [194]:
original_persons.sort(key=lambda d: d['search_name'])
print(len(original_persons))
original_persons

285


[{'search_name': 'Abraham Louis Breguet',
  'page_name': 'Abraham-Louis Breguet',
  'description': 'Swiss engineer and clockmaker',
  'Qcode': 'Q123679',
  'wiki_url': 'https://www.wikidata.org/wiki/Q123679'},
 {'search_name': 'Adam Jerzy Czartoryski',
  'page_name': 'Adam Czartoryski',
  'description': 'Polish nobleman, statesman, diplomat and author (1770-1861)',
  'Qcode': 'Q342889',
  'wiki_url': 'https://www.wikidata.org/wiki/Q342889'},
 {'search_name': 'Adam Mickiewicz',
  'page_name': 'Adam Mickiewicz',
  'description': 'Polish national poet, dramatist, essayist, publicist, translator, and political activist (1798-1855)',
  'Qcode': 'Q79822',
  'wiki_url': 'https://www.wikidata.org/wiki/Q79822'},
 {'search_name': 'Adam Łyszczyński',
  'page_name': 'Q99639218',
  'description': None,
  'Qcode': 'Q99639218',
  'wiki_url': 'https://www.wikidata.org/wiki/Q99639218'},
 {'search_name': 'Adolf Cichowski',
  'page_name': 'Adolphe Cichowski',
  'description': 'Polish soldier',
  'Qcode':

In [195]:
input_qcodes = [d['Qcode'] for d in original_persons]
input_qcodes

['Q123679',
 'Q342889',
 'Q79822',
 'Q99639218',
 'Q4505990',
 'Q77840132',
 'Q5738',
 'Q137819',
 'Q2082427',
 'Q3608671',
 'Q435375',
 'Q9145532',
 'Q20670032',
 'Q38337',
 'Q169150',
 'Q188697',
 'Q102194393',
 'Q4766898',
 'Q11686228',
 'Q11285375',
 'Q458156',
 'Q550766',
 'Q2074586',
 'Q64862',
 'Q106489731',
 'Q263091',
 'Q9157202',
 'Q561112',
 'Q9157627',
 'Q182011',
 'Q607998',
 'Q436726',
 'Q727056',
 'Q517132',
 'Q16054478',
 'Q766768',
 'Q713985',
 'Q2375473',
 'Q15975',
 'Q2896159',
 'Q9176254',
 'Q21338629',
 'Q2534918',
 'Q1630994',
 'Q159569',
 'Q1047458',
 'Q131552',
 'Q109810560',
 'Q62028861',
 'Q23060765',
 'Q663856',
 'Q55846558',
 'Q217068',
 'Q61319',
 'Q7322',
 'Q114366',
 'Q3021581',
 'Q9209760',
 'Q458688',
 'Q17551656',
 'Q5345972',
 'Q3051916',
 'Q8860780',
 'Q2658508',
 'Q1333348',
 'Q33477',
 'Q22248714',
 'Q55875031',
 'Q105330618',
 'Q57286',
 'Q269694',
 'Q46096',
 'Q533022',
 'Q213530',
 'Q41309',
 'Q951933',
 'Q15429707',
 'Q1441283',
 'Q242159',
 'Q

In [209]:
input_persons_wo_qcode = set(input) - {d['search_name'] for d in original_persons}
with open ("input_persons_wo_qcode.txt", "w") as f:
    f.write(str(input_persons_wo_qcode))
print(len(input_persons_wo_qcode))
input_persons_wo_qcode

114


{'Adam Crémieux',
 'Adam Towiański',
 'Agnès (Mlle)',
 'Aleksander Julian Hofman',
 'Aleksandra Karasowska',
 'Alojzy Janowicz (pułkownik)',
 'Amélie Franchomme',
 'Amélie Grille de Beuzelin',
 'Angelika Catalani',
 'Anna Maria Walewska (księżna)',
 'Anna Młokosiewicz',
 'Anna z Tańkowskich Paderewska',
 'Anne Elizabeth Louise Delaroche',
 'Antoni Żelisław Jędrzejewicz',
 'Antonina  Wilkońska',
 'August Clésinger',
 'Augustine Brault',
 'Augustine Marie Bertholdi',
 'Carl Helminger',
 'Carlotta Marliani',
 'Casimir Stanislas d Arpentigny',
 'Catherine de Souzzo (księżna)',
 'Charlotte Marliani',
 'Charlotte Rothschild (baronowa)',
 'Christian Rudolf Wessel',
 'Clotilde Elisabeth Villetard',
 'Cécile Franchomme',
 'Emil Jenike',
 'Ernest Canut',
 'Eugénie de Bonnechose',
 'Felicjan Mallefille',
 'Feliks Wodziński',
 'Fernand de Préaulx',
 'Fontana Juanna Camila',
 'Françoise (pokojowa George Sand)',
 'Friederich Kalkbrenner',
 'Fryderyk Bolesław Jędrzejewicz',
 'Fryderyk Skarbek (hrabia

# Humans associated with names from original_list ```(N levels of depth...)```

In [35]:
rdf_info = recursive_get_rdf_info(input_qcodes, level=3)
print(f"{len(rdf_info.keys())=}")

Done:  ['Q123679', 'Q342889', 'Q79822', 'Q99639218', 'Q4505990', 'Q77840132', 'Q5738', 'Q137819', 'Q2082427', 'Q3608671', 'Q435375', 'Q9145532', 'Q20670032', 'Q38337', 'Q169150', 'Q188697', 'Q102194393', 'Q4766898', 'Q11686228', 'Q11285375', 'Q458156', 'Q550766', 'Q2074586', 'Q64862', 'Q106489731', 'Q263091', 'Q9157202', 'Q561112', 'Q9157627', 'Q182011', 'Q607998', 'Q436726', 'Q727056', 'Q517132', 'Q16054478', 'Q766768', 'Q713985', 'Q2375473', 'Q15975', 'Q2896159', 'Q9176254', 'Q21338629', 'Q2534918', 'Q1630994', 'Q159569', 'Q1047458', 'Q131552', 'Q109810560', 'Q62028861', 'Q23060765', 'Q663856', 'Q55846558', 'Q217068', 'Q61319', 'Q7322', 'Q114366', 'Q3021581', 'Q9209760', 'Q458688', 'Q17551656', 'Q5345972', 'Q3051916', 'Q8860780', 'Q2658508', 'Q1333348', 'Q33477', 'Q22248714', 'Q55875031', 'Q105330618', 'Q57286', 'Q269694', 'Q46096', 'Q533022', 'Q213530', 'Q41309', 'Q951933', 'Q15429707', 'Q1441283', 'Q242159', 'Q356522', 'Q11698724', 'Q154353', 'Q157324', 'Q957041', 'Q523581', 'Q3136

In [217]:
# ^ czas działania recursive_get_rdf_info:
# rdf_info = recursive_get_rdf_info(input_qcodes, level=2)      # działało 3m 36s dla input-codes=['Q131861', 'Q255'] i zwróciło rdf_info dla 228 osób (104772 linijek słownika)
# rdf_info = recursive_get_rdf_info(input_qcodes, level=2)      # działało 41s dla input-codes=['Q131861'] i zwróciło rdf_info dla 36 osób (17376 linijek słownika)

# rdf_info = recursive_get_rdf_info(input_qcodes, level=3)      # działało 423m 35s dla input_codes z pliku .txt (399 osób (>1 słowo) z listów Chopina i Paderewskiego); len(rdf_info.keys())=13171; rdf_info 5 mln linijek;
                                                                # 180 MB json z indent=4; 117MB json w jednej linijce;

In [161]:
# import json
# with open("rdf_info.json", "w") as f:
#     json.dump(rdf_info, f)  # no idntent arg if you want smaller filesize

# rdf_info

In [211]:
qcodes_to_check = set()
for Q in rdf_info.keys():
    qcodes_to_check.add(Q)
    for rdf_tuple in rdf_info[Q]:
        if rdf_tuple[4] == "FactGrid item ID":
            continue
        object_url = rdf_tuple[2]
        qcode = object_url[object_url.rfind('Q'):]
        if len(qcode) > 1:
            if qcode[1:].isdigit():
                qcodes_to_check.add(qcode)
                if (qcode=='Q383232'):
                    print(Q, rdf_info[Q])
print(f"{len(qcodes_to_check) = }")

qcodes_to_check_split = []
n_size = 500  # splitting qcodes_to_check into lists of n_size (500) elements, because URI cannot be too large in function get_all_human_objects()
length = len(qcodes_to_check)
temp = []
for i, qcode in enumerate(qcodes_to_check):
    temp.append(qcode)
    if i%n_size == n_size-1:
        qcodes_to_check_split.append(temp)
        temp = []
    if i == length-1:
        if len(temp) > 0:
            qcodes_to_check_split.append(temp)
print(f"{len(qcodes_to_check_split) = }")

all_humans = []
n_lists = len(qcodes_to_check_split)
for i, lista in enumerate(qcodes_to_check_split):
    temp_list = get_all_human_objects(lista)
    all_humans += temp_list
# ^ 1m 30s dla len(qcodes_to_check) = 60829

len(qcodes_to_check) = 60829
len(qcodes_to_check_split) = 122
len(all_humans) = 29966


In [213]:

print(f"{len(all_humans) = }")
all_humans

len(all_humans) = 29966


[('Q57112', 'http://www.wikidata.org/entity/Q57112', 'Oswald Spengler'),
 ('Q61221',
  'http://www.wikidata.org/entity/Q61221',
  'Amalie Auguste of Bavaria'),
 ('Q63653',
  'http://www.wikidata.org/entity/Q63653',
  'Prince Alfons of Bavaria'),
 ('Q64235',
  'http://www.wikidata.org/entity/Q64235',
  'Princess Elisabeth of Saxe-Altenburg'),
 ('Q64704',
  'http://www.wikidata.org/entity/Q64704',
  'Prince Sigismund of Prussia'),
 ('Q65038',
  'http://www.wikidata.org/entity/Q65038',
  'Frederick III, Duke of Saxe-Gotha-Altenburg'),
 ('Q65648', 'http://www.wikidata.org/entity/Q65648', 'Charles Hallé'),
 ('Q66746', 'http://www.wikidata.org/entity/Q66746', 'Christiane Vulpius'),
 ('Q87575',
  'http://www.wikidata.org/entity/Q87575',
  'Philip Maurice, Count of Hanau-Münzenberg'),
 ('Q91211',
  'http://www.wikidata.org/entity/Q91211',
  'Gottfried Christoph Beireis'),
 ('Q110158', 'http://www.wikidata.org/entity/Q110158', 'Paul Homeyer'),
 ('Q116972',
  'http://www.wikidata.org/entity/Q116

# Creating neo4j graph

## connect to database

In [183]:
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "password"))
driver.verify_connectivity()
session = driver.session(database="wikidatadev")

## create Person nodes

In [184]:
all_humans_qcodes = []
for human in all_humans:
    qcode, wd_url, name = human
    all_humans_qcodes.append(qcode)
    try:
        cypher_query = f"""MERGE (:Person{{name: "{name}", wd_url: "{wd_url}"}})"""
        session.run(cypher_query)
    except CypherSyntaxError:  #  Karol Stanisław "Panie Kochanku" Radziwiłł (Q386484) 
        cypher_query = f"""MERGE (:Person{{name: '{name}', wd_url: '{wd_url}'}})"""
        session.run(cypher_query)
# ^ 2m16s dla len(all_humans) =~ 30_000 (powstałe z 399 osób z inputu)

## create other nodes, edges & connect to Person nodes

In [185]:
ERRORS = []
for qcode in rdf_info.keys():
    for t in rdf_info[qcode]:
        object_name, object_url = t[5].replace("'", "\\'"), t[2].replace("'", "\\'")

        object_label = "wdLabel"
        object_qcode = object_url[object_url.rfind("Q"):]
        if object_qcode in all_humans_qcodes: # in rdf_info.keys():  # in all_humans_qcodes: HERE
            object_label = "Person"
        else:
            cypher_query = f"""MERGE (:{object_label}{{name: '{object_name}', wd_url: '{object_url}'}})"""
            try:
                session.run(cypher_query)
            except Exception as e:
                ERRORS.append(("MERGE NODE", object_url, cypher_query, e))

        cypher_illegal_edgename_chars = ["'", ",", "(", ")", "-", ".", "/", "&", "’", "+", "+d'-", ":"]
        edge_name = t[4].replace(' ', '_').replace("–", "_")
        for c in cypher_illegal_edgename_chars:
            edge_name = edge_name.replace(c, "")
        edge_name = edge_name.upper()
        edge_url = t[1]

        subject_url = t[0].replace("'", "\\'")
        person_name = t[3].replace("'", "\\'")
        try:
            s = f"""
            MATCH (p:Person), (l:{object_label}) 
            WHERE p.name="{person_name}" AND p.wd_url="{subject_url}" AND l.name='{object_name}' AND l.wd_url='{object_url}'
            MERGE (p) - [:`{edge_name}` {{wd_url: '{edge_url}', source: 'wikidata'}}] -> (l)
            """
            session.run(s)
        except CypherSyntaxError:  # Karol Stanisław "Panie Kochanku" Radziwiłł (Q386484)
            s = f"""
            MATCH (p:Person), (l:{object_label}) 
            WHERE p.name='{person_name}' AND p.wd_url="{subject_url}" AND l.name='{object_name}' AND l.wd_url='{object_url}'
            MERGE (p) - [:`{edge_name}` {{wd_url: '{edge_url}', source: 'wikidata'}}] -> (l)
            """
            session.run(s)
        except Exception as e:
            ERRORS.append(("MERGE EDGE", object_url, s, e))

In [189]:
# ^ 45s dla len(rdf_info.keys()) = 36 len(all_humans) = 170
# ^ 936m 52s dla len(rdf_info.keys()) = 13171 len(all_humans) = 29966

In [220]:
ERRORS
# ^ lista ERRORS była pusta dla 936m 52s dla len(rdf_info.keys()) = 13171 len(all_humans) = 29966. To pewnie jakiś błąd jupytera(?), może za długo komórka działała. Jakeiś errory musiały się pojawić....

[]

## delete all nodes

In [180]:
# # DELETE ALL
# s="""MATCH (n)
# CALL { WITH n
# DETACH DELETE n
# } IN TRANSACTIONS;"""
# session.run(s)


## session close

In [175]:
# session.close()

## sample cypher queries

In [226]:
# UWAGA: nie można traktować uzyskanych wyników jako prawdę odzwierdciedlającą rzeczywistość; dane są niekompletne i pewnie niektóre nawet są nieprawdziwe.

# MATCH (n) WHERE n.name =~ ".*Chopin.*" RETURN n

# match (n:Person)-[e:OCCUPATION]-(s) WHERE s.name CONTAINS "composer" return n,s

# MATCH (p:Person)-[l:residence]-(s)
# WHERE p.name=~'.*Chopin' or p.name=~'.*Liszt'
# RETURN p, s

# MATCH (p:Person)-[l:RESIDENCE]-(s)
# WHERE p.name=~'.*Chopin' or p.name=~'.*Beethoven.*'
# RETURN p, s

# MATCH (p:Person)-[l:STUDENT_OF]-(s)
# WHERE p.name=~'.*Frédéric Chopin.*' or p.name=~'.*Franz Liszt.*'
# RETURN p, s

# Match(p:Person)-[e:date_of_birth]-(l:Label) return p, l

# match (n:Person) WHERE n.name CONTAINS "Bee" return n

# MATCH (p:Person)-[e:OCCUPATION]-(s:wdLabel) WHERE s.name =~ ".*singer.*" RETURN p,s

# MATCH w=(p:Person)-[e:INSTRUMENT]-(s:wdLabel)
# WITH s, COLLECT(w) as test
# ORDER BY SIZE(test) DESC
# RETURN s.name, SIZE(test)

# MATCH (p:Person)-[e:DATE_OF_BIRTH]-(d:wdLabel)
# WHERE datetime({year: 1770, month:7}) <= datetime(d.name) < datetime({year: 1771})
# WITH p, d
# ORDER BY d.name DESC
# RETURN p, d, p.name, d.name

# MATCH (p:Person)-[e:HANDEDNESS]-(s:wdLabel), (p)-[i:INSTRUMENT]-(s2:wdLabel)
# WHERE s.name =~ '.*left.*' 
# RETURN *  

# MATCH (p:Person)-[e:HANDEDNESS]-(s:wdLabel)
# WHERE s.name =~ '.*left.*' 
# RETURN *  


# Komentarze
Zbiór niezbyt dobrze uporządkowanych myśli i nieścisłych wyrażeń *(informacje dla mnie, po jakimś czasie i tak przestałem aktualizować tę sekcję)*

Co zrobiłem:
* uporządkowałem kod
* odrzuciłem pojedyncze nazwy z listy osób, np. "Pierre"
* ```get_all_triples_and_qcodes()``` dodałem opcję zostawienia tych objects, które nie są Qcodami w wikidata (różne literały, często są to jakieś id w muzeach czy innych zbiorach danych, wcześniej to odrzuciłem bo wydało mi się nieprzydatne) 
* ^ ale wd_url będzie wtedy literałem, a nie urlem w grafie. Nie jest to chyba duży problem i może tak zostać. ( ale TODO do poprawy)
* dla krawędzi: uppercase nazw i dodanie atrybutu ```source: 'wikidata' ```
* dla każdego subject z listy sprawdzenie wszystkich powiązanych objects i zapisanie qcodów tych obiektów, które są ```instance_of``` --> ```human```
* ```scrape_qcodes(list, humans_only=False, first_only=False)``` dodałem sposób na dostanie Qcode tylko pierwszego znalezionego człowieka na wikidata
* zrezygnowałem z atrybutu ```description``` dla scrapeowanych qcodów (w grafie)
* rekurencyjne pobieranie info o ludziach połączonych z ludźmi z inputu

Kilka problematycznych kwestii:
* osoba, o którą nam chodzi nie jest pierwszym wynikiem na liście wyszukiwania w wikidata
* niektórych osób z input listy nie wyszukuje w wikdata. Jak się usunie dopiski w nawiasach, to już tak, np. dla "Józef Michał Poniatowski (książę)". Z 399 osób w input list, zostało 285 z Qcodami (ale nie wiem, czy poprawnymi). TODO do poprawy
* czy kontrolujemy manualnie poprawność osób dodanych z wikidata? (chodzi o osoby z input_list)
* biblioteka SPARQLWrapper nie działa dobrze, zwraca często zmultiplikowane wyniki i spowalnia działanie programu
* osoby, które są na najniższym poziome (liście drzewa) nie mają żadnych atrybutów (trójek rdf), bo kiedyś ten łańcuch trzeba skończyć. Pytanie w jaki sposób. 
* w grafie prawdopodobnie będą błędy, np. randomowe osoby nazywające się tak samo jak np. te wspomniane w listach, a tych wspomnianych nie będzie w grafie
* będą sobie pływały randomowe węzły osób w tym grafie, np. Q383232 (Afiat Yuris Wirawan, gracz badmintona) bo np Rachmaninov ma "FactGrid item ID": Q383232
* ^ EDIT:   naprawiłem to dla tego specyficznego przypadku krawędzi "FactGrid item ID", u nich wszystkie ID są postaci takiej jak Qcody na wikidata
* ^         ale mogą się pojawić inne randomowe osoby i przez to nie będzie tworzony węzęł z obiektem o nazwie np. Q123 i nie będzie krawędzi między subjectem(osobą) a obiektem/randomową osobą
* a może nie brać osób z listów Paderewskiego, a zostawić tylko osoby z listów Chopina?

['Etienne',
 'Perrichet',
 'Pierre',
 'Alexis',
 'Grandville',
 'Rafael',
 'Voltaire']

Granville https://www.wikidata.org/wiki/Q744442 ???

Voltaire https://www.wikidata.org/wiki/Q9068 ???

Pierre służący Fryderyka Chopina ???



--- na razie robię sobie przerwę od tego projektu, muszę zacząć robić pracę inżynierską i mam też inny projekt

# TODO

* Zbyt długo trwa robienie grafu. Może zrobienie csv i potem wczytywanie csv do neo4j będzie szybsze niż pojedyncze robienie węzłów i krawędzi. Zrobić indeksy, zwiększyć limity pamięci w neo4j desktop, zrobić CSV do krawędzi i LOAD CSV, CALL IN TRANSACTIONS. Do zrobienia.
* Niektórych osób z input listy nie wyszukuje w wikdata. Jak się usunie dopiski w nawiasach, to już tak, np. dla "Józef Michał Poniatowski (książę)". Do zrobienia.
* w np. DATE_OF_BIRTH są jakieś śmieci czasami, np. http://www.wikidata.org/.well-known/genid/c9a8b2f22783c60e6bf383200d9285f7. Pewnie można by w "create other nodes, edges & connect to Person nodes" sprawdzać wartość dla qcodów dat np date_of_birth, date_of_costam. Do zrobienia.
* A może by tak oznaczać osoby (węzeł :Person), które nie mają atrybutów z wikidata (czyli w grafie powiązań z węzłami wdLabel) jakąś inną etykietą, lub jakimś atrybutem, np. "leaf"/"no_further_info"/"no_". Są to "osoby-liście", na których kończy się rekurencyjne pobieranie info o ludziach. ({all_humans} - {rdf_info.keys()}).       
* Biblioteka SPARQLWrapper nie działa dobrze, zwraca często zmultiplikowane wyniki i spowalnia działanie programu, można spróbować użyć innej biblioteki lub samemu napisać.
* Może nie brać osób z listów Paderewskiego, a zostawić tylko osoby z listów Chopina?
* [EDIT] [nieważne, neo4j browser nie wyświetla krawędzi czasami] czemu wielu kompozytorów nie ma OCCUPATION -> composer? np. Robert Schumann... Wygląda na to, że czasamiu niektóre krawędzie się nie tworzą, zbadać przyczynę i naprawić. Do zrobienia.

* Na githuba wrzucic kod i na discorda linka do https://query.wikidata.org/ jakby ktos chcial sie zajac pobieraniem innych danych nie dotyczacych osob itp.

** TYLKO POMYSŁ: sporo czasu by trzeba poświęcić na sprawdzanie czy pierszy scrape'owany human z wikidata search to osoba której chcemy (czyli w jakiś sposób powiązana z Chopinem/osobami na grafie).
